# Embeddings pipeline
## Results
- 2 MB per embedding

## Problems
- In 152 cases, len(domain_sequence) doesn't match len(domain_end - domain_start) + 1
- Warning: Found 164 entries where 'sequence' is NOT a substring of 'protein_sequence' at the specified domain boundaries.
- Solution: Drop all sequences where the domain doesnt match the sequence ...
- 834 embeddings have 38 GB in total (~45 MB per embedding)

In [16]:

import os

import pandas as pd

output_dir = "../data/embeddings"
protein_embeddings_dir = os.path.join(output_dir, "protein_embeddings")
domain_embeddings_dir = os.path.join(output_dir, "domain_embeddings")
os.makedirs(protein_embeddings_dir, exist_ok=True)
os.makedirs(domain_embeddings_dir, exist_ok=True)
dataset = pd.read_csv("../data/subset_with_protein_mapping.csv")
dataset

,class,architecture,topology,homology,domain_id,s35,s60,s95,s100,s100_count,length,resolution,sequence,homology_path,protein_sequence,domain_start,domain_end,note
0,1,10,8,10,1oksA00,5,1,1,1,1,51,1.80,MASRSVIRSIIKSSRLEEDRKRYLMTLLDDIKGANDLAKFHQMLVK...,1.10.8.10,MASRSVIRSIIKSSRLEEDRKRYLMTLLDDIKGANDLAKFHQMLVK...,1,56,NaN
1,1,10,8,10,4dbgB02,36,1,1,1,1,61,2.71,PYVLEMVAELAGQQDPGLGAFSCQEARRAWLDRHGNLDEAVEECVR...,1.10.8.10,GPLGSRQDKMREEGLQLVSMIREGEAAGACPEEIFSALQYSGTEVP...,55,115,NaN
2,1,10,8,10,4un2B00,11,1,1,1,1,43,1.51,GSPEERYEHQLRQLNDMGFFDFDRNVAALRRSGGSVQGALDSLLNGDV,1.10.8.10,GSPEERYEHQLRQLNDMGFFDFDRNVAALRRSGGSVQGALDSLLNGDV,1,48,NaN
3,1,10,8,10,1oaiA00,1,1,1,1,1,59,1.00,PTLSPEQQEMLQAFSTQSGMNLEWSQKCLQDNNWDYTRSAQAFTHL...,1.10.8.10,PTLSPEQQEMLQAFSTQSGMNLEWSQKCLQDNNWDYTRSAQAFTHL...,1,59,NaN
4,1,10,8,10,1g3iW02,46,1,1,1,1,18,3.41,LVRQQEIAKNREELKQKA,1.10.8.10,MSEMTPREIVSELDQHIIGQADAKRAVAIALRNRWRRMQLQEPLRH...,110,142,"121(E;A), 122(E;R), 123(L;A), 124(K;E), 125(Q;D)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,100,10,20,2yzsA01,2,1,1,1,1,80,2.00,GRVYYINSHGTLSRHENTLRFENAEVKKDIPVEDVEEIFVFAELSL...,3.100.10.20,GRVYYINSHGTLSRHENTLRFENAEVKKDIPVEDVEEIFVFAELSL...,1,80,NaN
996,3,100,10,20,7cr6D01,9,1,1,1,1,82,3.72,TLYLTQPDAVLSKKHEAFHVALKQEDGSWKKQPIPAQTLEDIVLLG...,3.100.10.20,GASGSGTGSGSMSTLYLTQPDAVLSKKHEAFHVALKQEDGSWKKQP...,14,98,NaN
997,3,100,10,20,4n06A01,5,1,1,1,1,81,2.40,GGMRLVVDGFGKYLGIENGLIVVKEKGKALRKVRPEDLKQVLIIGK...,3.100.10.20,GGMRLVVDGFGKYLGIENGLIVVKEKGKALRKVRPEDLKQVLIIGK...,1,82,NaN
998,3,100,10,20,8d3mC01,8,1,1,1,1,88,3.41,KKLLNTLYVTQPDTYLSLDGDNVVLLKEQEKLGRLPLHNLEAIVGF...,3.100.10.20,MKKLLNTLYVTQPDTYLSLDGDNVVLLKEQEKLGRLPLHNLEAIVG...,2,89,NaN


In [17]:
print("Original dataset shape:", dataset.shape)

# Create a boolean mask to identify rows with substring mismatches
mismatch_mask = []
for index, row in dataset.iterrows():
    domain_start = int(row['domain_start'])
    domain_end = int(row['domain_end'])
    prot_sequence = row['sequence']
    protein_sequence = row['protein_sequence']

    # Extract the potential substring from protein_sequence (adjusting for 1-based start)
    # Ensure domain_start - 1 is not negative
    start_idx = max(0, domain_start - 1)

    # Handle cases where domain_end might exceed protein_sequence length
    end_idx = min(domain_end, len(protein_sequence))

    expected_substring = protein_sequence[start_idx:end_idx]

    if prot_sequence != expected_substring:
        mismatch_mask.append(True)  # Mark for dropping
    else:
        mismatch_mask.append(False)  # Keep

# Convert the list to a pandas Series for boolean indexing
mismatch_series = pd.Series(mismatch_mask, index=dataset.index)

# Get the domain_ids of the rows to be dropped for logging
domain_ids_to_drop = dataset.loc[mismatch_series, 'domain_id'].tolist()

if domain_ids_to_drop:
    print(f"\nIdentified {len(domain_ids_to_drop)} entries with substring mismatches to be dropped:")
    print("Domain IDs to be dropped:", domain_ids_to_drop)
else:
    print("\nNo substring mismatches found. No rows will be dropped based on this criterion.")

# Drop the rows where mismatch_series is True
dataset_cleaned = dataset[~mismatch_series].copy()  # Use .copy() to avoid SettingWithCopyWarning

print("\nDataset shape after dropping mismatched domain positions:", dataset_cleaned.shape)

# You can now continue working with dataset_cleaned
# For example, if you want to overwrite the original dataset variable:
dataset = dataset_cleaned
print("Updated dataset variable shape:", dataset.shape)

Original dataset shape: (1000, 18)

Identified 164 entries with substring mismatches to be dropped:
Domain IDs to be dropped: ['1g3iW02', '7aj9A01', '8db4F01', '5mmjo00', '4lwsA00', '2az0B00', '3ziuB01', '1qu3A04', '1tazA00', '1so2A00', '2o8bA04', '1wb9A03', '1e3mB04', '3thxA03', '2o8bB03', '4h8aA01', '1nxuA01', '1wtjB01', '3uoeA01', '1vbiA01', '2x06A01', '1z2iA01', '2g8yB01', '3i0pA01', '1r8eA02', '2zhgA00', '2xhiA02', '1pu6A01', '1ornA02', '4unfA01', '5kn8A01', '1keaA01', '4uobA01', '3n0uC01', '3n5nX01', '4modA00', '7rsjA01', '2wxfA01', '7bi4A01', '3ls8B01', '2b5iD01', '1vs9S01', '5dm7R00', '1wv4B00', '1rfeA01', '1nl1A00', '1pkyC03', '1vf7A02', '5oomW00', '1g29103', '2hzmH00', '3l81A02', '4en2A02', '2vglM02', '4iknA02', '3l81A01', '2pr9A01', '5wrkA01', '4iknA01', '1vrbD01', '7zccA01', '3purA02', '7wffb01', '2cirA00', '4xjwA02', '2bf6A01', '7q4oC01', '3v0sA00', '4i3gA01', '6zvhC01', '3j7yp00', '6ks6B02', '7xokN01', '1kp8A02', '4xcgA02', '6ks6e02', '7ylyh01', '6ks6G02', '3ruvA02', '8bm

In [11]:
from src.embedding import ProtT5Embeddings

model_name = "Rostlab/prot_t5_xl_half_uniref50-enc"

protT5Embeddings = ProtT5Embeddings(model_name=model_name)


Loading model: Rostlab/prot_t5_xl_half_uniref50-enc
Model loaded successfully


In [12]:
from tqdm import tqdm
import torch

batch_size = 16
all_embeddings = {}

for i in tqdm(range(0, len(dataset), batch_size), desc="Processing batches"):
    batch_df = dataset[i:i + batch_size]
    prot_sequences = batch_df['protein_sequence'].tolist()
    dom_sequences = batch_df['sequence'].tolist()
    domain_boundaries = batch_df[['domain_start', 'domain_end']].values.tolist()
    domain_ids = batch_df['domain_id'].tolist()
    sequence_lengths_calculated = [len(seq) for seq in prot_sequences]

    print(f"Max sequence length : {max(sequence_lengths_calculated)}, "
          f"Min sequence length : {min(sequence_lengths_calculated)}")

    if prot_sequences:
        embeddings = protT5Embeddings.embed_sequences(prot_sequences)
        for j, domain_id in enumerate(domain_ids):
            prot_sequence = prot_sequences[j]
            protein_embedding = embeddings[j, :len(prot_sequence)].cpu()
            domain_start = domain_boundaries[j][0] - 1
            domain_end = domain_boundaries[j][1]
            domain_embedding = protein_embedding[domain_start:domain_end]

            protein_embedding_path = os.path.join(protein_embeddings_dir, f"{domain_id}.pt")
            domain_embedding_path = os.path.join(domain_embeddings_dir, f"{domain_id}.pt")

            #torch.save(protein_embedding, protein_embedding_path)
            torch.save(domain_embedding, domain_embedding_path)

            print(f"\n--- Domain ID: {domain_id} ---")
            print(f"Domain Boundaries (start, end): ({domain_start}, {domain_end})")
            print(
                f"Len prot_embedding({len(protein_embedding)}) == Len Dom_Sequence({len(prot_sequence)}): {len(protein_embedding) == len(prot_sequence)}")
            print(
                f"Len dom_embedding({len(domain_embedding)}) == Len Dom_Sequence({len(dom_sequences[j])}): {len(domain_embedding) == len(dom_sequences[j])}")




Processing batches:   0%|          | 0/28 [00:00<?, ?it/s]

Max sequence length : 650, Min sequence length : 82

--- Domain ID: 4uzgA01 ---
Domain Boundaries (start, end): (0, 166)
Len prot_embedding(292) == Len Dom_Sequence(292): True
Len dom_embedding(166) == Len Dom_Sequence(166): True

--- Domain ID: 3v10A02 ---
Domain Boundaries (start, end): (157, 307)
Len prot_embedding(321) == Len Dom_Sequence(321): True
Len dom_embedding(150) == Len Dom_Sequence(150): True

--- Domain ID: 8begA01 ---
Domain Boundaries (start, end): (5, 166)
Len prot_embedding(650) == Len Dom_Sequence(650): True
Len dom_embedding(161) == Len Dom_Sequence(161): True

--- Domain ID: 2x9zA01 ---
Domain Boundaries (start, end): (0, 149)
Len prot_embedding(262) == Len Dom_Sequence(262): True
Len dom_embedding(149) == Len Dom_Sequence(149): True

--- Domain ID: 8begA03 ---
Domain Boundaries (start, end): (337, 486)
Len prot_embedding(650) == Len Dom_Sequence(650): True
Len dom_embedding(149) == Len Dom_Sequence(149): True

--- Domain ID: 3uxfA02 ---
Domain Boundaries (start, 

Processing batches:   4%|▎         | 1/28 [00:41<18:28, 41.05s/it]


--- Domain ID: 4eiuA01 ---
Domain Boundaries (start, end): (0, 112)
Len prot_embedding(249) == Len Dom_Sequence(249): True
Len dom_embedding(112) == Len Dom_Sequence(112): True

--- Domain ID: 5aq0B00 ---
Domain Boundaries (start, end): (0, 82)
Len prot_embedding(82) == Len Dom_Sequence(82): True
Len dom_embedding(82) == Len Dom_Sequence(82): True

--- Domain ID: 4fxtG01 ---
Domain Boundaries (start, end): (0, 95)
Len prot_embedding(202) == Len Dom_Sequence(202): True
Len dom_embedding(95) == Len Dom_Sequence(95): True

--- Domain ID: 3hn5A01 ---
Domain Boundaries (start, end): (0, 100)
Len prot_embedding(215) == Len Dom_Sequence(215): True
Len dom_embedding(100) == Len Dom_Sequence(100): True
Max sequence length : 1014, Min sequence length : 82

--- Domain ID: 1uwyA02 ---
Domain Boundaries (start, end): (296, 403)
Len prot_embedding(426) == Len Dom_Sequence(426): True
Len dom_embedding(107) == Len Dom_Sequence(107): True

--- Domain ID: 4iykA01 ---
Domain Boundaries (start, end): (0,

Processing batches:   7%|▋         | 2/28 [04:07<59:57, 138.37s/it]

...


--- Domain ID: 1iqqA00 ---
Domain Boundaries (start, end): (0, 200)
Len prot_embedding(200) == Len Dom_Sequence(200): True
Len dom_embedding(200) == Len Dom_Sequence(200): True
Max sequence length : 984, Min sequence length : 167

--- Domain ID: 4dw4B00 ---
Domain Boundaries (start, end): (0, 167)
Len prot_embedding(167) == Len Dom_Sequence(167): True
Len dom_embedding(167) == Len Dom_Sequence(167): True

--- Domain ID: 1iybA00 ---
Domain Boundaries (start, end): (0, 208)
Len prot_embedding(208) == Len Dom_Sequence(208): True
Len dom_embedding(208) == Len Dom_Sequence(208): True

--- Domain ID: 3t0oA00 ---
Domain Boundaries (start, end): (0, 238)
Len prot_embedding(238) == Len Dom_Sequence(238): True
Len dom_embedding(238) == Len Dom_Sequence(238): True

--- Domain ID: 2pqxA00 ---
Domain Boundaries (start, end): (0, 245)
Len prot_embedding(245) == Len Dom_Sequence(245): True
Len dom_embedding(245) == Len Dom_Sequence(245): True

--- Domain ID: 5an9B01 ---
Domain Boundaries (start, end

Processing batches:  96%|█████████▋| 27/28 [1:23:05<02:43, 163.29s/it]


--- Domain ID: 3godC01 ---
Domain Boundaries (start, end): (16, 104)
Len prot_embedding(328) == Len Dom_Sequence(328): True
Len dom_embedding(88) == Len Dom_Sequence(88): True
Max sequence length : 347, Min sequence length : 315


Processing batches: 100%|██████████| 28/28 [1:23:20<00:00, 178.59s/it]


--- Domain ID: 2yzsA01 ---
Domain Boundaries (start, end): (0, 80)
Len prot_embedding(315) == Len Dom_Sequence(315): True
Len dom_embedding(80) == Len Dom_Sequence(80): True

--- Domain ID: 7cr6D01 ---
Domain Boundaries (start, end): (13, 98)
Len prot_embedding(336) == Len Dom_Sequence(336): True
Len dom_embedding(85) == Len Dom_Sequence(85): True

--- Domain ID: 4n06A01 ---
Domain Boundaries (start, end): (0, 82)
Len prot_embedding(347) == Len Dom_Sequence(347): True
Len dom_embedding(82) == Len Dom_Sequence(82): True

--- Domain ID: 8d3mC01 ---
Domain Boundaries (start, end): (1, 89)
Len prot_embedding(343) == Len Dom_Sequence(343): True
Len dom_embedding(88) == Len Dom_Sequence(88): True

--- Domain ID: 3godD01 ---
Domain Boundaries (start, end): (0, 103)
Len prot_embedding(328) == Len Dom_Sequence(328): True
Len dom_embedding(103) == Len Dom_Sequence(103): True


In [18]:
dataset.to_csv("../data/subset_with_protein_embedding.csv", index=False)

In [19]:
len(dataset)

836

In [20]:
embedding = torch.load("../data/embeddings/domain_embeddings/1a3wB03.pt")

In [21]:
embedding.shape

torch.Size([102, 1024])